# Telecom Churn Business Case
# Preprocess data and create train, validation and test data for logistic regression and tensor friendly format
We have data from an Audiobook app. Logically, it relates only to the audio versions of books. Each customer in the database has made a purchase at least once, that's why he/she is in the database. We want to create a machine learning algorithm based on our available data that can predict if a customer will buy again from the Audiobook company.Link to this kaggle dataset is :https://www.kaggle.com/jpacse/datasets-for-churn-telecom?select=cell2celltrain.csv 


# Import important Packages and extract data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
sns.set()

C:\Users\samar\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
raw_data = pd.read_csv('cell2celltrain.csv')

In [3]:
data_no_custID = raw_data.drop(['CustomerID','HandsetPrice','ServiceArea'],axis=1)

In [4]:
data_no_custID['Churn'] = data_no_custID['Churn'].map({'Yes':1 ,'No':0})

In [5]:
data_no_missing_value=data_no_custID.dropna(axis=0)

# Remove extreme values

In [6]:
q = data_no_missing_value['MonthlyRevenue'].quantile(0.99)
p = data_no_missing_value['MonthlyRevenue'].quantile(0.01)
data1 = data_no_missing_value[data_no_missing_value['MonthlyRevenue']<q]
data1  = data_no_missing_value[data_no_missing_value['MonthlyRevenue']>p] 

In [7]:
q = data1['MonthlyMinutes'].quantile(0.99)
p = data1['MonthlyMinutes'].quantile(0.01)
data2 = data1[data1['MonthlyRevenue']<q]
data2  = data1[data1['MonthlyRevenue']>p] 

In [8]:
q = data2['TotalRecurringCharge'].quantile(0.99)
p = data2['TotalRecurringCharge'].quantile(0.01)
data3 = data2[data2['TotalRecurringCharge']<q]
data3  = data2[data2['TotalRecurringCharge']>p] 

In [9]:
data_unscaled_all = data3.copy()
data_unscaled_all = data_unscaled_all.reset_index(drop=True)
target_churn = data_unscaled_all['Churn']

# Balance the Dataset

In [10]:
num_one_targets = int(np.sum(target_churn))
zero_targets_counter = 0

indices_to_remove = []

for i in range(target_churn.shape[0]):
    if target_churn[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
              indices_to_remove.append(i)
 
#data_unscaled_equal_priors = np.delete(data_unscaled_all, indices_to_remove, axis = 0)
data_unscaled_prior = data_unscaled_all.drop(indices_to_remove)
data_targets_prior = data_unscaled_prior['Churn']
data_unscaled_inputs = data_unscaled_prior.drop(['Churn'],axis=1)


In [11]:
data_unscaled_all = data_unscaled_prior.copy()

# Segerate categorical and continuous variables

In [12]:
data_category_only = data_unscaled_all[['ChildrenInHH', 'HandsetRefurbished',
       'HandsetWebCapable', 'TruckOwner', 'RVOwner', 'Homeownership',
       'BuysViaMailOrder', 'RespondsToMailOffers', 'OptOutMailings',
       'NonUSTravel', 'OwnsComputer', 'HasCreditCard', 'NewCellphoneUser',
       'NotNewCellphoneUser', 'OwnsMotorcycle', 
       'MadeCallToRetentionTeam', 'CreditRating', 'PrizmCode',
       'Occupation', 'MaritalStatus','AdjustmentsToCreditRating','ReferralsMadeBySubscriber','RetentionOffersAccepted','RetentionCalls','Churn']]

In [13]:
data_unscale_continuous = data_unscaled_all.drop(['ChildrenInHH', 'HandsetRefurbished',
       'HandsetWebCapable', 'TruckOwner', 'RVOwner', 'Homeownership',
       'BuysViaMailOrder', 'RespondsToMailOffers', 'OptOutMailings',
       'NonUSTravel', 'OwnsComputer', 'HasCreditCard', 'NewCellphoneUser',
       'NotNewCellphoneUser', 'OwnsMotorcycle', 
       'MadeCallToRetentionTeam', 'CreditRating', 'PrizmCode',
       'Occupation', 'MaritalStatus', 'AdjustmentsToCreditRating',
       'ReferralsMadeBySubscriber', 'RetentionOffersAccepted',
       'RetentionCalls', 'Churn'],axis =1 )

In [14]:
data_unscale_continuous.columns.values

array(['MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge',
       'DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls',
       'PercChangeMinutes', 'PercChangeRevenues', 'DroppedCalls',
       'BlockedCalls', 'UnansweredCalls', 'CustomerCareCalls',
       'ThreewayCalls', 'ReceivedCalls', 'OutboundCalls', 'InboundCalls',
       'PeakCallsInOut', 'OffPeakCallsInOut', 'DroppedBlockedCalls',
       'CallForwardingCalls', 'CallWaitingCalls', 'MonthsInService',
       'UniqueSubs', 'ActiveSubs', 'Handsets', 'HandsetModels',
       'CurrentEquipmentDays', 'AgeHH1', 'AgeHH2', 'IncomeGroup'],
      dtype=object)

In [15]:
data_input_categorical = data_category_only.iloc[:,:-1]
data_input_categorical

,ChildrenInHH,HandsetRefurbished,HandsetWebCapable,TruckOwner,RVOwner,Homeownership,BuysViaMailOrder,RespondsToMailOffers,OptOutMailings,NonUSTravel,...,OwnsMotorcycle,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus,AdjustmentsToCreditRating,ReferralsMadeBySubscriber,RetentionOffersAccepted,RetentionCalls
0,No,No,Yes,No,No,Known,Yes,Yes,No,No,...,No,Yes,1-Highest,Suburban,Professional,No,0,0,0,1
1,Yes,No,No,No,No,Known,Yes,Yes,No,No,...,No,No,4-Medium,Suburban,Professional,Yes,0,0,0,0
2,Yes,No,No,No,No,Unknown,No,No,No,No,...,No,No,3-Good,Town,Crafts,Yes,0,0,0,0
3,No,No,Yes,No,No,Known,Yes,Yes,No,No,...,No,No,4-Medium,Other,Other,No,0,0,0,0
4,No,No,No,No,No,Known,Yes,Yes,No,No,...,No,No,1-Highest,Other,Professional,Yes,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46206,No,Yes,Yes,No,No,Known,No,No,No,Yes,...,No,No,2-High,Other,Professional,No,0,0,0,0
46208,No,No,Yes,No,No,Unknown,No,No,No,No,...,No,No,3-Good,Other,Other,No,0,0,0,0
46210,No,No,Yes,No,No,Unknown,No,No,No,No,...,No,No,4-Medium,Other,Other,Unknown,0,0,0,0
46212,No,No,Yes,No,No,Known,Yes,Yes,No,No,...,No,No,1-Highest,Other,Professional,No,0,0,0,0


In [16]:
data_target = data_targets_prior.copy()

# Standardize continuous input variables

In [17]:
from sklearn.preprocessing import StandardScaler
churn_scaler = StandardScaler()
churn_scaler.fit(data_unscale_continuous)
scaled_inputs = churn_scaler.transform(data_unscale_continuous)

In [18]:
data_inputs_continuous = pd.DataFrame(scaled_inputs, columns=['MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge',
       'DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls',
       'PercChangeMinutes', 'PercChangeRevenues', 'DroppedCalls',
       'BlockedCalls', 'UnansweredCalls', 'CustomerCareCalls',
       'ThreewayCalls', 'ReceivedCalls', 'OutboundCalls', 'InboundCalls',
       'PeakCallsInOut', 'OffPeakCallsInOut', 'DroppedBlockedCalls',
       'CallForwardingCalls', 'CallWaitingCalls', 'MonthsInService',
       'UniqueSubs', 'ActiveSubs', 'Handsets', 'HandsetModels',
       'CurrentEquipmentDays', 'AgeHH1', 'AgeHH2', 'IncomeGroup'])

In [19]:
data_input_categorical =data_input_categorical.reset_index(drop=True)
data_inputs_continuous=data_inputs_continuous.reset_index(drop=True)

# Combine categorical and standardize input variables

In [20]:
data_inputs = pd.concat([data_input_categorical,data_inputs_continuous],axis=1)

# Create dummmy for category variables

In [21]:
chilrenInHH_dummy = pd.get_dummies(data_inputs['ChildrenInHH'],drop_first=True)


In [22]:
data_inputs['Chiilren'] = chilrenInHH_dummy

In [23]:
data_inputs = data_inputs.drop(['ChildrenInHH'],axis=1)

In [24]:
Handset_dummy = pd.get_dummies(data_inputs['HandsetRefurbished'],drop_first=True)

In [25]:
data_inputs['RefurbishedHandset'] = Handset_dummy


In [26]:
data_inputs = data_inputs.drop(['HandsetRefurbished'],axis=1)

In [27]:
HandsetWebCapable_dummy = pd.get_dummies(data_inputs['HandsetWebCapable'],drop_first=True)
data_inputs['HSWebCapable'] = HandsetWebCapable_dummy
data_inputs = data_inputs.drop(['HandsetWebCapable'],axis=1)

In [28]:
truckOwner_dummy = pd.get_dummies(data_inputs['TruckOwner'],drop_first=True)
data_inputs['TruckOwn'] = truckOwner_dummy
data_inputs = data_inputs.drop(['TruckOwner'],axis=1)

In [29]:
RVOwn_dummy = pd.get_dummies(data_inputs['RVOwner'],drop_first=True)
data_inputs['RVOwn'] = RVOwn_dummy
data_inputs = data_inputs.drop(['RVOwner'],axis=1)

In [30]:
homeowner_dummy =  pd.get_dummies(data_inputs['Homeownership'],drop_first=True)


In [31]:
data_inputs['homeown'] = homeowner_dummy
data_inputs = data_inputs.drop(['Homeownership'],axis=1)

In [32]:
mail_dummy = pd.get_dummies(data_inputs['BuysViaMailOrder'],drop_first=True)
data_inputs['BuyThrMail'] = mail_dummy
data_inputs = data_inputs.drop(['BuysViaMailOrder'],axis=1)

In [33]:
mailres_dummy = pd.get_dummies(data_inputs['RespondsToMailOffers'],drop_first=True)
data_inputs['MailResponse'] = mailres_dummy
data_inputs = data_inputs.drop(['RespondsToMailOffers'],axis=1)

In [34]:
mailoptout_dummy = pd.get_dummies(data_inputs['OptOutMailings'],drop_first=True)
data_inputs['MailListOptout'] = mailoptout_dummy
data_inputs = data_inputs.drop(['OptOutMailings'],axis=1)

In [35]:
NonUStravel_dummy = pd.get_dummies(data_inputs['NonUSTravel'],drop_first=True)
data_inputs['NonUSTrip'] = NonUStravel_dummy
data_inputs = data_inputs.drop(['NonUSTravel'],axis=1)

In [36]:
Owncomputer_dummy = pd.get_dummies(data_inputs['OwnsComputer'],drop_first=True)
data_inputs['OwnComp'] = Owncomputer_dummy
data_inputs = data_inputs.drop(['OwnsComputer'],axis=1)

In [37]:
HasCC_dummy = pd.get_dummies(data_inputs['HasCreditCard'],drop_first=True)
data_inputs['HasCC'] = HasCC_dummy
data_inputs = data_inputs.drop(['HasCreditCard'],axis=1)

In [38]:
NewCellPhoneUser_dummy = pd.get_dummies(data_inputs['NewCellphoneUser'],drop_first=True)
data_inputs['NewCellUser'] = NewCellPhoneUser_dummy
data_inputs = data_inputs.drop(['NewCellphoneUser'],axis=1)

In [39]:
data_inputs = data_inputs.drop(['NotNewCellphoneUser'],axis=1)

In [40]:
OwnMotorCycle_dummy = pd.get_dummies(data_inputs['OwnsMotorcycle'],drop_first=True)
data_inputs['HasMotorCycle'] = OwnMotorCycle_dummy
data_inputs = data_inputs.drop(['OwnsMotorcycle'],axis=1)

In [41]:
CalltoReten_dummy = pd.get_dummies(data_inputs['MadeCallToRetentionTeam'],drop_first=True)
data_inputs['CallToRetentionTeam'] = CalltoReten_dummy
data_inputs = data_inputs.drop(['MadeCallToRetentionTeam'],axis=1)

In [42]:
data_inputCheckpoint = data_inputs.copy()

In [43]:
CreditRating_dummy = pd.get_dummies(data_inputs['CreditRating'],drop_first=True)
data_inputs = pd.concat([data_inputs,CreditRating_dummy],axis=1)
data_inputs = data_inputs.drop(['CreditRating'],axis=1)

In [44]:
PrizmCode_dummy = pd.get_dummies(data_inputs['PrizmCode'],drop_first=True)
data_inputs = pd.concat([data_inputs,PrizmCode_dummy],axis=1)
data_inputs = data_inputs.drop(['PrizmCode'],axis=1)

In [45]:
occupation_dummy = pd.get_dummies(data_inputs['Occupation'],drop_first=True)
data_inputs = pd.concat([data_inputs,occupation_dummy],axis=1)
data_inputs = data_inputs.drop(['Occupation'],axis=1)

In [46]:
data_no_ms = data_inputs.drop(['MaritalStatus'],axis=1)

In [47]:
data_no_ms['AdjustmentInCreditRating'] = np.where(data_no_ms['AdjustmentsToCreditRating'] > 0,1,0)

In [48]:
data_no_ms = data_no_ms.drop(['AdjustmentsToCreditRating'],axis=1)

In [49]:
data_no_ms['CustomerMakesRef'] = np.where(data_no_ms['ReferralsMadeBySubscriber'] > 0,1,0)

In [50]:
data_no_ms = data_no_ms.drop(['ReferralsMadeBySubscriber'],axis=1)

In [51]:
data_no_ms['AcceptedRetentionOffer'] = np.where(data_no_ms['RetentionOffersAccepted'] > 0,1,0)

In [52]:
data_no_ms = data_no_ms.drop(['RetentionOffersAccepted'],axis=1)

In [53]:
data_no_ms['RetentionCalls'] = np.where(data_no_ms['RetentionCalls'] > 0,1,0)

# Finalize input and target data

In [54]:
pd.set_option('display.max_columns', None)
data_no_ms.head()
data_clean = data_no_ms.copy()
data_clean = data_clean.reset_index(drop=True)
data_target = data_target.reset_index(drop=True)
data_target = pd.DataFrame(data_target, columns=['Churn'])
input_target_combine = pd.concat([data_target,data_clean],axis=1)

# Reshuffle data

In [55]:
from sklearn.utils import shuffle
shuffled_input_target_combine = shuffle(input_target_combine)

# Save final preprocessed data as csv for logit regression

In [56]:
shuffled_input_target_combine.to_csv('TelecomChurn_logistic.csv',index=False)

In [57]:
shuffled_targets = pd.DataFrame(shuffled_input_target_combine['Churn'],columns=['Churn'])
shuffled_inputs = shuffled_input_target_combine.drop(['Churn'],axis=1)

In [58]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

Churn    10450
dtype: int64 20902 Churn    0.499952
dtype: float64
Churn    1253
dtype: int64 2612 Churn    0.479709
dtype: float64
Churn    1361
dtype: int64 2614 Churn    0.520658
dtype: float64


# Save preprocessed data as npz tensor friendly format for neural network 

In [59]:
np.savez('Telecom_churn_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Telecom_churn_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Telecom_churn_data_test', inputs=test_inputs, targets=test_targets)

In [60]:
train_inputs.shape

(20902, 65)